### Your very own neural network

In this notebook we're going to build a neural network using naught but pure numpy and steel nerves. It's going to be fun, I promise!

<img src="frankenstein.png" style="width:20%">

In [ ]:
import sys
sys.path.append("..")
import tqdm_utils
import download_utils

from typing import List

In [ ]:
# use the preloaded keras datasets and models
download_utils.link_all_keras_resources()

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(42)
# np.seterr(all='raise')

Here goes our main class: a layer that can do .forward() and .backward() passes.

In [ ]:
class Layer:
    """
    A building block. Each layer is capable of performing two things:
    
    - Process input to get output:           output = layer.forward(input)
    
    - Propagate gradients through itself:    grad_input = layer.backward(input, grad_output)
    
    Some layers also have learnable parameters which they update during layer.backward.
    """
    def __init__(self):
        """Here you can initialize layer parameters (if any) and auxiliary stuff."""
        # A dummy layer does nothing
        pass
    
    def forward(self, layer_input):
        """
        Takes input data of shape [batch, input_units], returns output data [batch, output_units]
        """
        # A dummy layer just returns whatever it gets as input.
        return layer_input

    def backward(self, layer_input, grad_output):
        """
        Performs a backpropagation step through the layer, with respect to the given input.
        
        To compute loss gradients w.r.t input, you need to apply chain rule (backprop):
        
        d loss / d x  = (d loss / d layer) * (d layer / d x)
        
        Luckily, you already receive d loss / d layer as input, so you only need to multiply it by d layer / d x.
        
        If your layer has parameters (e.g. dense layer), you also need to update them here using d loss / d layer
        """
        # The gradient of a dummy layer is precisely grad_output, but we'll write it more explicitly
        num_units = layer_input.shape[1]
        
        d_layer_d_input = np.eye(num_units)
        
        return np.dot(grad_output, d_layer_d_input) # chain rule

### The road ahead

We're going to build a neural network that classifies MNIST digits. To do so, we'll need a few building blocks:
- Dense layer - a fully-connected layer, $f(X)=W \cdot X + \vec{b}$
- ReLU layer (or any other nonlinearity you want)
- Loss function - crossentropy
- Backprop algorithm - a stochastic gradient descent with backpropageted gradients

Let's approach them one at a time.


### Nonlinearity layer

This is the simplest layer you can get: it simply applies a nonlinearity to each element of your network.

In [ ]:
class ReLU(Layer):
    def __init__(self):
        """ReLU layer simply applies elementwise rectified linear unit to all inputs"""
        super().__init__()

    def forward(self, layer_input):
        """Apply elementwise ReLU to [batch, input_units] matrix"""
        relu = np.vectorize(lambda x: np.maximum(0, x))
        return relu(layer_input)

    def backward(self, layer_input, grad_output):
        """Compute gradient of loss w.r.t. ReLU input"""
        relu_grad = layer_input > 0
        return grad_output*relu_grad        

In [ ]:
# some tests

from util import eval_numerical_gradient

def test_relu():
    x = np.linspace(-1,1,10*32).reshape([10,32])
    l = ReLU()
    grads = l.backward(x,np.ones([10,32])/(32*10))
    numeric_grads = eval_numerical_gradient(lambda _x: l.forward(_x).mean(), x)
    assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0),\
        "gradient returned by your layer does not match the numerically computed gradient"

test_relu()

#### Instant primer: lambda functions

In python, you can define functions in one line using the `lambda` syntax: `lambda param1, param2: expression`

For example: `f = lambda x, y: x+y` is equivalent to a normal function:

```
def f(x,y):
    return x+y
```
For more information, click [here](http://www.secnetix.de/olli/Python/lambda_functions.hawk).    

### Dense layer

Now let's build something more complicated. Unlike nonlinearity, a dense layer actually has something to learn.

A dense layer applies affine transformation. In a vectorized form, it can be described as:
$$f(X)= W \cdot X + \vec b $$

Where 
* X is an object-feature matrix of shape [batch_size, num_features],
* W is a weight matrix [num_features, num_outputs] 
* and b is a vector of num_outputs biases.

Both W and b are initialized during layer creation and updated each time backward is called.

In [ ]:
class WeightsInitialiser:
    def get_weights(self, input_units, output_units):
        weights = np.random.randn(input_units, output_units) * 0.01
        biases = np.zeros(output_units)
        return weights, biases

class Dense(Layer):
    def __init__(self, input_units, output_units, weights_init: WeightsInitialiser = None, learning_rate=0.1):
        """
        A dense layer is a layer which performs a learned affine transformation:
        f(x) = <W*x> + b
        """
        super().__init__()

        self.learning_rate = learning_rate

        # initialize weights with small random numbers. We use normal initialization, 
        # but surely there is something better. Try this once you got it working: http://bit.ly/2vTlmaJ
        if weights_init is None:
            weights_init = WeightsInitialiser()
        self.weights, self.biases = weights_init.get_weights(input_units, output_units)

    def forward(self, layer_input):
        """
        Perform an affine transformation:
        f(x) = <W*x> + b
        
        input shape: [batch, input_units]
        output shape: [batch, output units]
        """
        return np.dot(layer_input, self.weights) + self.biases

    def backward(self, layer_input, grad_output):

        # compute (d f / d x) = (d f / d dense) * (d dense / d x)
        # where (d dense / d x) = weights transposed
        d_f_d_dense = grad_output
        d_dense_d_x = self.weights.T
        grad_input = np.dot(d_f_d_dense, d_dense_d_x)

        # compute gradient w.r.t. weights and biases
        grad_weights = np.dot(layer_input.T, d_f_d_dense)
        grad_biases = np.sum(grad_output, axis=0).transpose()

        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape

        # Here we perform a stochastic gradient descent step. 
        # Later on, you can try replacing that with something better.
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases

        return grad_input

### Testing the dense layer

Here we have a few tests to make sure your dense layer works properly. You can just run them, get 3 "well done"s and forget they ever existed.

... or not get 3 "well done"s and go fix stuff. If that is the case, here are some tips for you:
* Make sure you compute gradients for W and b as __sum of gradients over batch__, not mean over gradients. Grad_output is already divided by batch size.
* If you're debugging, try saving gradients in class fields, like "self.grad_w = grad_w" or print first 3-5 weights. This helps debugging.
* If nothing else helps, try ignoring tests and proceed to network training. If it trains alright, you may be off by something that does not affect network training.

In [ ]:
def test_dense_forward():
    l = Dense(128, 150)

    assert -0.05 < l.weights.mean() < 0.05 and 1e-3 < l.weights.std() < 1e-1,\
        "The initial weights must have zero mean and small variance. "\
        "If you know what you're doing, remove this assertion."
    assert -0.05 < l.biases.mean() < 0.05, "Biases must be zero mean. Ignore if you have a reason to do otherwise."

    # To test the outputs, we explicitly set weights with fixed values. DO NOT DO THAT IN ACTUAL NETWORK!
    l = Dense(3,4)

    x = np.linspace(-1,1,2*3).reshape([2,3])
    l.weights = np.linspace(-1,1,3*4).reshape([3,4])
    l.biases = np.linspace(-1,1,4)

    assert np.allclose(l.forward(x), np.array([[ 0.07272727,  0.41212121,  0.75151515,  1.09090909],
                                               [-0.90909091,  0.08484848,  1.07878788,  2.07272727]]))
    print("Well done!")

test_dense_forward()

In [ ]:
# To test the grads, we use gradients obtained via finite differences

from util import eval_numerical_gradient

def test_dense_backward():
    x = np.linspace(-1,1,10*32).reshape([10,32])
    l = Dense(32,64,learning_rate=0)

    numeric_grads = eval_numerical_gradient(lambda _x: l.forward(_x).sum(), x)
    grads = l.backward(x,np.ones([10,64]))

    assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0), "input gradient does not match numeric grad"
    print("Well done!")

test_dense_backward()

In [ ]:
#test gradients w.r.t. params
def compute_out_given_wb(w, b):
    l = Dense(32,64,learning_rate=1)
    l.weights = np.array(w)
    l.biases = np.array(b)
    x = np.linspace(-1,1,10*32).reshape([10,32])
    return l.forward(x)
    
def compute_grad_by_params(w, b):
    l = Dense(32,64,learning_rate=1)
    l.weights = np.array(w)
    l.biases = np.array(b)
    x = np.linspace(-1,1,10*32).reshape([10,32])
    l.backward(x,np.ones([10,64]) / 10.)
    return w - l.weights, b - l.biases

def test_dense_gds():
    w,b = np.random.randn(32,64), np.linspace(-1,1,64)

    numeric_dw = eval_numerical_gradient(lambda _w: compute_out_given_wb(_w, b).mean(0).sum(), w)
    numeric_db = eval_numerical_gradient(lambda _b: compute_out_given_wb(w, _b).mean(0).sum(), b)
    grad_w, grad_b = compute_grad_by_params(w, b)

    assert np.allclose(numeric_dw, grad_w, rtol=1e-3, atol=0), "weight gradient does not match numeric weight gradient"
    assert np.allclose(numeric_db, grad_b, rtol=1e-3, atol=0), "bias gradient does not match numeric bias gradient"
    print("Well done!")

test_dense_gds()

### The logits activation

In [ ]:
glob_layer_input = None

class Logits(Layer):
    def __init__(self):
        """Logits layer simply calculates logits for layer outputs"""
        super().__init__()

    @staticmethod
    def _sigmoid( _input):
        return 1 / (1 + np.exp(- _input))

    def forward(self, layer_input):
        """Apply elementwise logits to [batch, input_units] matrix"""
        return self._sigmoid(layer_input)

    def backward(self, layer_input, grad_output):
        """Compute gradient of loss w.r.t. logits input"""
        sigm = self._sigmoid(layer_input)
        d_S_d_x = sigm * (1 - sigm)
        return grad_output * d_S_d_x

### The loss function

Since we want to predict probabilities, it would be logical for us to define softmax nonlinearity on top of our network and compute loss given predicted probabilities. However, there is a better way to do so.

If you write down the expression for crossentropy as a function of softmax logits (a), you'll see:

$$ loss = - log \space {e^{a_{correct}} \over {\underset i \sum e^{a_i} } } $$

If you take a closer look, ya'll see that it can be rewritten as:

$$ loss = - a_{correct} + log {\underset i \sum e^{a_i} } $$

It's called Log-softmax and it's better than naive log(softmax(a)) in all aspects:
* Better numerical stability
* Easier to get derivative right
* Marginally faster to compute

So why not just use log-softmax throughout our computation and never actually bother to estimate probabilities.

Here you are! We've defined the both loss functions for you so that you could focus on neural network part.

In [ ]:
def softmax_crossentropy_with_logits(logits, reference_answers):
    """Compute crossentropy from logits[batch,n_classes] and ids of correct answers"""
    logits_for_answers = logits[np.arange(len(logits)), reference_answers]

    xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))

    return xentropy

def grad_softmax_crossentropy_with_logits(logits, reference_answers):
    """Compute crossentropy gradient from logits[batch,n_classes] and ids of correct answers"""
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)), reference_answers] = 1

    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)

    return (- ones_for_answers + softmax) / logits.shape[0]

In [ ]:
def test_softmax():
    logits = np.linspace(-1,1,500).reshape([50,10])
    answers = np.arange(50)%10

    softmax_crossentropy_with_logits(logits, answers)
    grads = grad_softmax_crossentropy_with_logits(logits, answers)
    numeric_grads = eval_numerical_gradient(lambda l: softmax_crossentropy_with_logits(l, answers).mean(), logits)

    assert np.allclose(numeric_grads, grads, rtol=1e-3, atol=0), \
        "The reference implementation has just failed. Someone has just changed the rules of math."

test_softmax()

### Full network

Now let's combine what we've just built into a working neural network. As we announced, we're gonna use this monster to classify handwritten digits, so let's get them loaded.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from preprocessed_mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=True)

plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title("Label: %i"%y_train[i])
    plt.imshow(X_train[i].reshape([28,28]),cmap='gray')

We'll define network as a list of layers, each applied on top of previous one. In this setting, computing predictions and training becomes trivial.

In [ ]:
def create_simple_network() -> List[Layer]:
    network = []
    network.append(Dense(X_train.shape[1], 100))
    network.append(ReLU())
    network.append(Dense(100, 200))
    network.append(ReLU())
    network.append(Dense(200, 10))
    network.append(Logits())
    return network

In [ ]:
def forward(network: List[Layer], x):
    """
    Compute activations of all network layers by applying them sequentially.
    Return a list of activations for each layer. 
    Make sure last activation corresponds to network logits.
    """
    activations = []
    layer_input = x

    for layer in network:
        output = layer.forward(layer_input)
        activations.append(output)
        layer_input = output

    assert len(activations) == len(network)
    return activations

def predict(network: List[Layer], x):
    """
    Compute network predictions.
    """
    logits = forward(network, x)[-1]
    return logits.argmax(axis=-1)

def train(network: List[Layer], x, y):
    """
    Train your network on a given batch of X and y.
    You first need to run forward to get all layer activations.
    Then you can run layer.backward going from last to first layer.
    
    After you called backward for all layers, all Dense layers have already made one gradient step.
    """

    # Get the layer activations
    layer_activations = forward(network, x)
    layer_inputs = [x] + layer_activations[:-1]  #layer_input[i] is an input for network[i]

    assert len(network) == len(layer_inputs),\
        "Number of layers {} should match the number of layer inputs {}".format(len(network), len(layer_inputs))

    # Compute the loss and the initial gradient
    logits = layer_activations[-1]
    loss = softmax_crossentropy_with_logits(logits, y)
    loss_grad = grad_softmax_crossentropy_with_logits(logits, y)

    # perform backpropagation
    for layer, layer_input in zip(reversed(network), reversed(layer_inputs)):
        loss_grad = layer.backward(layer_input, loss_grad)

    return np.mean(loss)

Instead of tests, we provide you with a training loop that prints training and validation accuracies on every epoch.

If your implementation of forward and backward are correct, your accuracy should grow from 90~93% to >97% with the default network.

### Training loop

As usual, we split data into minibatches, feed each such minibatch into the network and update weights.

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in tqdm_utils.tqdm_notebook_failsafe(range(0, len(inputs) - batchsize + 1, batchsize)):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
from IPython.display import clear_output

In [ ]:
def train_loop(network, epoch_number, _x_train, _y_train, _x_val, _y_val):
    clear_output()
    train_log = []
    val_log = []

    for epoch in range(epoch_number):
        for x_batch, y_batch in iterate_minibatches(_x_train, _y_train, batchsize=32, shuffle=True):
            train(network, x_batch, y_batch)

        train_log.append(np.mean(predict(network, _x_train) == _y_train))
        val_log.append(np.mean(predict(network, _x_val) == _y_val))

        clear_output()
        print("Epoch", epoch)
        print("Train accuracy:", train_log[-1])
        print("Val accuracy:", val_log[-1])
        plt.plot(train_log, label='train accuracy')
        plt.plot(val_log, label='val accuracy')
        plt.legend(loc='best')
        plt.grid()
        plt.show()

In [ ]:
simple_network = create_simple_network()
train_loop(simple_network, 25, X_train, y_train, X_val, y_val)

### Peer-reviewed assignment

Congradulations, you managed to get this far! There is just one quest left undone, and this time you'll get to choose what to do.


#### Option I: initialization
* Implement Dense layer with Xavier initialization as explained [here](http://bit.ly/2vTlmaJ)

To pass this assignment, you must conduct an experiment showing how xavier initialization compares to default initialization on deep networks (5+ layers).


#### Option II: regularization
* Implement a version of Dense layer with L2 regularization penalty: when updating Dense Layer weights, adjust gradients to minimize

$$ Loss = Crossentropy + \alpha \cdot \underset i \sum {w_i}^2 $$

To pass this assignment, you must conduct an experiment showing if regularization mitigates overfitting in case of abundantly large number of neurons. Consider tuning $\alpha$ for better results.

#### Option III: optimization
* Implement a version of Dense layer that uses momentum/rmsprop or whatever method worked best for you last time.

Most of those methods require persistent parameters like momentum direction or moving average grad norm, but you can easily store those params inside your layers.

To pass this assignment, you must conduct an experiment showing how your chosen method performs compared to vanilla SGD.

### General remarks
_Please read the peer-review guidelines before starting this part of the assignment._

In short, a good solution is one that:
* is based on this notebook
* runs in the default course environment with Run All
* its code doesn't cause spontaneous eye bleeding
* its report is easy to read.

_Formally we can't ban you from writing boring reports, but if you bored your reviewer to death, there's noone left alive to give you the grade you want._


### Bonus assignments

As a bonus assignment (no points, just swag), consider implementing Batch Normalization ([guide](https://gab41.lab41.org/batch-normalization-what-the-hey-d480039a9e3b)) or Dropout ([guide](https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5)). Note, however, that those "layers" behave differently when training and when predicting on test set.

* Dropout:
  * During training: drop units randomly with probability __p__ and multiply everything by __1/(1-p)__
  * During final predicton: do nothing; pretend there's no dropout
  
* Batch normalization
  * During training, it substracts mean-over-batch and divides by std-over-batch and updates mean and variance.
  * During final prediction, it uses accumulated mean and variance.

### Xavier initialization

Of the three possible additional tasks, I chose to improve the initialization of initial weights of neurons.
Based on the description of the initialization methods
([link 1](https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79)),
([link 2](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf))
([link 3](https://arxiv.org/pdf/1502.01852.pdf)).
I used Xavier initialization for the final layer with the symmetric activation function - sigmoid.
For the rest of the layers - with asymmetric ReLu activation - I used Kaiming initialization of weights.

I created a network of 7 layers and trained it during 25 epochs.
For comparison, I created the network with same topology, but initialized the weights using the standard distribution.

The results of the experiment demonstrate that the network initialized using the Xavier/Kaiming methods
has quite good quality - the accuracy in the control sample was 97.25%.
Although the  network tends to overfit.

The network with weights initialized by the standard distribution just doesn't train -
gradients become vanished in the very first epoch.
Depending on the initial conditions, it is still possible to train such a network.
But in my experiments, the gradients vanished no later than in the 5th epoch.

In [ ]:
class XavierWeightsInitialiser(WeightsInitialiser):
    def get_weights(self, input_units, output_units):
        weights = np.random.uniform(-1, 1, (input_units, output_units)) * np.sqrt(6 / (input_units + output_units))
        biases = np.zeros(output_units)
        return weights, biases


class KaimingWeightsInitialiser(WeightsInitialiser):
    def get_weights(self, input_units, output_units):
        weights = np.random.randn(input_units, output_units) * np.sqrt(2 / input_units)
        biases = np.zeros(output_units)
        return weights, biases


def create_deep_network(default_weights: bool) -> List[Layer]:
    if default_weights:
        weights_init_asymmetric = WeightsInitialiser()
        weights_init_symmetric = WeightsInitialiser()
    else:
        weights_init_asymmetric = KaimingWeightsInitialiser()
        weights_init_symmetric = XavierWeightsInitialiser()

    network = []
    network.append(Dense(X_train.shape[1], 100, weights_init=weights_init_asymmetric))
    network.append(ReLU())
    network.append(Dense(100, 200, weights_init=weights_init_asymmetric))
    network.append(ReLU())
    network.append(Dense(200, 300, weights_init=weights_init_asymmetric))
    network.append(ReLU())
    network.append(Dense(300, 500, weights_init=weights_init_asymmetric))
    network.append(ReLU())
    network.append(Dense(500, 250, weights_init=weights_init_asymmetric))
    network.append(ReLU())
    network.append(Dense(250, 100, weights_init=weights_init_asymmetric))
    network.append(ReLU())
    network.append(Dense(100, 10, weights_init=weights_init_symmetric))
    network.append(Logits())
    return network


In [ ]:
default_weights_network = create_deep_network(True)
train_loop(default_weights_network, 10, X_train, y_train, X_val, y_val)

In [ ]:
smart_weights_network = create_deep_network(False)
train_loop(smart_weights_network, 25, X_train, y_train, X_val, y_val)
